In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')
    #'/home/achenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [2]:
%matplotlib
#inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import Recordings
import signale,trajectory
import extras.trees
import os,sys
import fnmatch
import colormaps as cmaps
import seaborn as sns
import scipy.signal as scsig
import seaborn as sns
import random
from statsmodels.nonparametric.kernel_density import KDEMultivariate,
import pandas as pd
from scipy.io import savemat

Using matplotlib backend: Qt5Agg


/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/NeuroTools/analysis.py:34: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to u'Qt5Agg' by the following code:
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages

In [51]:
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
def my_shuffle(array):
        random.shuffle(array)
        return array
def locate(pattern, root=os.curdir):
    '''
    Locate all files matching supplied filename pattern in and below
        supplied root directory.
    Parameters
    ----------
        pattern : string
            A string representing pattern you want to look for in the path.
        root : string
            contains the path address you want to look trough!
    Returns
    ----------
        Array of strings consist of two column, first contains the path and second the file names that matched 
        the desired pattern.
    '''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            yield [path,filename]
def ncs_loader(fileName):
    ncs_dtype = np.dtype([
        ('qTimeStamps'  ,     '<u8'),
        ('ChannelNumber'  , '<u4'),
        ('SampleFreq',      '<u4'),
        ('NumValidSamples', '<u4'),
        ('Samples'  ,       '<i2',(512,)),
    ])
    return np.memmap(fileName,dtype=ncs_dtype,offset=(16 * 2**10))
def nvt_loader(filename):
    """ 
    Memory map the Neuralynx .nvt format
    Fields
    -------
    swstx
    swid
    sw_data_size
    qTimeStamps       Cheetah timestamp for this record. This value is in microseconds.
    dwpoints          Points with the color bitfield values for this record.This is a 400
                      element array.  See Video Tracker in reference. 
    sncrc
    dnextracted_x     Extracted X location of the target being tracked. 
    dnextracted_y     Extracted Y location of the target being tracked.
    dnextracted_angle The calculated head angle in degrees clockwise from the positive Y
                      axis. Zero will be assigned if angle tracking is disabled.
    dnTargets         Colored targets using the samebitfield format used by the dwPoints array.
                      Instead of transitions, the bitfield indicates the colors that make up 
                      each particular target and the center point of that target.  This is a 50
                      element array sorted by size from largest (index 0) to smallest(index 49).
                      A target value of 0 means that no target is present in thatindex location.
                      See Video Tracker Bitfield Information in reference. 
    
    Reference:
    ----------
        http://neuralynx.com/software/NeuralynxDataFileFormats.pdf
    """ 
    nev_dtype = np.dtype([
        ('swstx'              , '<i2'),
        ('swid'               , '<i2'),
        ('sw_data_size'       , '<i2'),
        ('qTimeStamps'        , '<u8'),
        ('dwPoints'           , '<u4',(400,)),
        ('sncrc'              , '<i2'),
        ('dnextracted_x'      , '<i4'),
        ('dnextracted_y'      , '<i4'),
        ('dnextracted_angle'  , '<i4'),
        ('dnTargets'          , '<i4',(50,)),
    ])
    return np.memmap(filename, dtype=nev_dtype, mode='readwrite',
       offset=(16 * 2**10))
def nev_loader(filename):
    """ Memory map the Neuralynx .nev format """ 
    nev_dtype = np.dtype([
        ('nstx'           , '<i2'),
        ('npkt_id'        , '<i2'),
        ('npkt_data_size' , '<i2'),
        ('qTimeStamps'    , '<u8'),
        ('nevent_id'      , '<i2'),
        ('nttl'           , '<i2'),
        ('ncrc'           , '<i2'),
        ('ndummy1'        , '<i2'),
        ('ndummy2'        , '<i2'),
        ('dnExtra'        , '<i4',(8,)),
        ('String'         , '<S128')
    ])
    return np.memmap(filename, dtype=nev_dtype, mode='r',
       offset=(16 * 2**10))
###################################################### initialization
###################################################### Parameters
expType = 'real'
prefix = 'TT'
suffix = ''
noSpeck = False
onlyRunning = True
showHeadDir = False
saveFigs = True
useRecommended = False
TTName = '.t'

# initialize in order to make them available globally
spikes=[]
ID=-1
stList=signale.spikezugList(t_start=None, t_stop=None, dims=[2])
eventData = None

cwd=os.getcwd()


###################################################### functions

def getData(folderName):
    global spikes, ID, eventData

    if os.path.isdir(folderName):
        dirList=os.listdir(folderName)
        os.chdir(folderName)
    else:
        dirList = [folderName]
    for item in sorted(dirList):
        if os.path.isfile(item):
            if (TTName.__class__ == list and item in TTName) or\
                    (TTName.__class__ == str and item.endswith(suffix+'.t') and item.startswith(prefix)):
                print 'loading', item , 'from folder: '+'/'.join(folderName.split('/')[-3:])
                spikes = signale.load_tFile(item, showHeader=False)
                ID += 1
                stList.__setitem__(ID, spikes)
                stList.addTags(ID, file=item, dir=folderName)
            
        elif os.path.isdir(item):
            getData(item)
    os.chdir('..')


In [52]:
WorkPath = '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/'
experimentFolder = '/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/'

In [53]:
Nslices = 1 #number of slices that a theta cycle should be cut
animal = experimentFolder.split('/')[-3]
dday = experimentFolder.split('_')[-1][:-1]
print animal+'-----'+dday
parrentFolder = os.path.join(experimentFolder,'parrentData/')
getData(parrentFolder)
if Nslices==1:
    tt = pkl.load(open(parrentFolder+'thBins.npArr','rb'))
    bb = 100
else:
    tt = pkl.load(open(parrentFolder+'4thBins.npArr','rb'))
    bb = 25
pre,track,post  = [stList.time_slice(item[0],item[-1]) for item in tt[:3]]
ExperimentSessions = [pre,track,post]
corrMatrices = []
Qs = []
eigVals = []
eigVecs = []
Ps = []
for ii,item in enumerate(ExperimentSessions):
    print ii
    Q,C,l,V,P = item.PCA(bb)
    corrMatrices.append(C)
    Qs.append(Q)
    eigVals.append(l)
    eigVecs.append(V)
    Ps.append(P)
            
#####PCA with time bins defined by theta periods
ThetaCorrMatrices = []
ThetaQs = []
ThetaEigVals = []
ThetaEigVecs = []
ThetaPs = []
for ii,item in enumerate(ExperimentSessions):
    print ii
    Q,C,l,V,P = item.PCA(bins=tt[ii])
    ThetaCorrMatrices.append(C)
    ThetaQs.append(Q)
    ThetaEigVals.append(l)
    ThetaEigVecs.append(V)
    ThetaPs.append(P)

N = stList.id_list.size*1.0
lMax = []
lMin = []
TlMax = []
TlMin = []
for q,Tq in zip(Qs,ThetaQs):
    lMax.append(np.math.pow(1 + np.math.sqrt(N/q.size),2))
    lMin.append(np.math.pow(1 - np.math.sqrt(N/q.size),2))
    TlMax.append(np.math.pow(1 + np.math.sqrt(N/Tq.size),2))
    TlMin.append(np.math.pow(1 - np.math.sqrt(N/Tq.size),2))
TW_limit =  np.math.pow(N,-0.6666666666)
#print lMax,lMin,TW_limit
##################################################################
f,axarr = pl.subplots(2,3,figsize=[15,10],sharey=True)
axarr[0,0].scatter(range(eigVals[0].size),eigVals[0],s=25)
axarr[0,0].axhline(lMax[0]+TW_limit,color = 'r',linestyle='--')
axarr[0,1].scatter(range(eigVals[1].size),eigVals[1],s=25)
axarr[0,1].axhline(lMax[1]+TW_limit,color = 'r',linestyle='--')
axarr[0,2].scatter(range(eigVals[2].size),eigVals[2],s=25)
axarr[0,2].axhline(lMax[2]+TW_limit,color = 'r',linestyle='--')
############
axarr[1,0].scatter(range(ThetaEigVals[0].size),ThetaEigVals[0],s=75)
axarr[1,0].axhline(TlMax[0]+TW_limit,color = 'r',linestyle='--')
axarr[1,1].scatter(range(eigVals[1].size),ThetaEigVals[1],s=25)
axarr[1,1].axhline(TlMax[1]+TW_limit,color = 'r',linestyle='--')
axarr[1,2].scatter(range(eigVals[2].size),ThetaEigVals[2],s=25)
axarr[1,2].axhline(TlMax[2]+TW_limit,color = 'r',linestyle='--')
######################################################################
Gamma = []
TGamma = []
for ii in range(3):
    G = []
    TG = []
    for jj in range(int(N)):
        G.append(eigVecs[1][jj].T.dot(corrMatrices[ii]).dot(eigVecs[1][jj]))
        TG.append(ThetaEigVecs[1][jj].T.dot(ThetaCorrMatrices[ii]).dot(ThetaEigVecs[1][jj]))
    Gamma.append(np.array(G))
    TGamma.append(np.array(TG))
print np.where(eigVals[2]>lMax[2]+TW_limit)
print np.where(ThetaEigVals[2]>TlMax[2]+TW_limit)

Rat3906-----Day1
loading TT12_01.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_02.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_03.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_04.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_05.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_06.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_07.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_08.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_09.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_10.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_11.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_12.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_13.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loading TT12_14.t from folder: LinearTrack_05-31-16_Day1/parrentData/
loa

In [ ]:
pl.plot(np.diff(tt[0]))

In [54]:
sigN = 1
f,(ax1,ax2,ax3,ax4) = pl.subplots(1,4,figsize=[30,10])
ax1.scatter(eigVals[1][sigN:],Gamma[0][sigN:],s=10)
ax1.scatter(ThetaEigVals[1][sigN:],TGamma[0][sigN:],c='r',s=10)
ax2.scatter(eigVals[1][sigN:],Gamma[1][sigN:],s=10)
ax2.scatter(ThetaEigVals[1][sigN:],TGamma[1][sigN:],c='r',s=10)
ax3.scatter(eigVals[1][sigN:],Gamma[2][sigN:],s=10)
ax3.scatter(ThetaEigVals[1][sigN:],TGamma[2][sigN:],c='r',s=10)
ax4.scatter(Gamma[0][sigN:],Gamma[2][sigN:],s=100)
ax4.scatter(TGamma[0][sigN:],TGamma[2][sigN:],c='r',s=100,alpha=0.7)
ax1.plot([0.51,1.85],[0.51,1.85],'k--', lw=1)
ax2.plot([0.51,1.85],[0.51,1.85],'k--', lw=1)
ax3.plot([0.51,1.85],[0.51,1.85],'k--', lw=1)
ax4.plot([0.51,1.85],[0.51,1.85],'k--', lw=1)
ax1.set_ylabel('Avg Reactivation PRE')
ax1.set_xlabel('Signal EigVals LinearTrack')
ax3.set_ylabel('Avg Reactivation POST')
ax2.set_xlabel('Signal EigVals LinearTrack')
ax3.set_xlabel('Signal EigVals LinearTrack')
ax4.set_ylabel('Avg Reactivation POST')
ax4.set_xlabel('Avg Reactivation PRE')
ax1.set_title('Sleep PRE')
ax3.set_title('Sleep POST')
ax4.set_title('PRE vs. POST')

Text(0.5,1,u'PRE vs. POST')

In [55]:
GmSh = []
TGmSh = []
for ss in range(1500):
    
    evSH = []
    TevSH = []
    for ii,ev in enumerate(eigVecs[1]):
        shuffledIDS  = my_shuffle(stList.id_list)
        evSH.append(ev[shuffledIDS])
        TevSH.append(ThetaEigVecs[1][ii][shuffledIDS])
    for ii in range(3):
        G = []
        TG = []
        for jj in range(int(N)):
            cm = corrMatrices[ii]
            #cmSH = cm[:,shuffledIDS]
            #cmSH = cmSH[shuffledIDS,:]
            G.append(evSH[jj].T.dot(cm).dot(evSH[jj]))
            TG.append(TevSH[jj].T.dot(ThetaCorrMatrices[ii]).dot(TevSH[jj]))
        GmSh.append(np.array(G))
        TGmSh.append(np.array(TG))

In [56]:
idfake = my_shuffle(range(10))
print idfake
culSH = np.arange(100).reshape([10,10])[:,idfake]
culSH[idfake,:]

[6, 2, 1, 0, 4, 9, 7, 5, 3, 8]


array([[66, 62, 61, 60, 64, 69, 67, 65, 63, 68],
       [26, 22, 21, 20, 24, 29, 27, 25, 23, 28],
       [16, 12, 11, 10, 14, 19, 17, 15, 13, 18],
       [ 6,  2,  1,  0,  4,  9,  7,  5,  3,  8],
       [46, 42, 41, 40, 44, 49, 47, 45, 43, 48],
       [96, 92, 91, 90, 94, 99, 97, 95, 93, 98],
       [76, 72, 71, 70, 74, 79, 77, 75, 73, 78],
       [56, 52, 51, 50, 54, 59, 57, 55, 53, 58],
       [36, 32, 31, 30, 34, 39, 37, 35, 33, 38],
       [86, 82, 81, 80, 84, 89, 87, 85, 83, 88]])

In [57]:
from scipy.stats import  linregress
sns.set_style('ticks')
sigN=0
f,(ax,bx) = pl.subplots(1,2,figsize=[20,10])
Gscatt = []
TGscatt = []
for ii in np.arange(0,4447,3):
    for jj in range(sigN,eigVals[1].size):
        Gscatt.append([GmSh[ii][jj],GmSh[ii+2][jj]])
        TGscatt.append([TGmSh[ii][jj],TGmSh[ii+2][jj]])
Gscatt = np.array(Gscatt)
TGscatt = np.array(TGscatt)
print linregress(Gscatt[:,0],Gscatt[:,1])
print linregress(TGscatt[:,0],TGscatt[:,1])
#sns.kdeplot(Gscatt,ax=ax,shade=True,shade_lowest=False,n_levels=20)
#sns.kdeplot(TGscatt,ax=bx,shade=True,shade_lowest=False)
ax.plot([0.81,1.85],[0.81,1.85],'k--', lw=1)
ax.scatter(Gamma[0][sigN:],Gamma[2][sigN:],marker='x',c='k',s=50,zorder=20)
bx.plot([0.81,1.85],[0.81,1.85],'k--', lw=1)
bx.scatter(TGamma[0][sigN:],TGamma[2][sigN:],marker='x',c='k',s=50,zorder=20)
#pl.scatter(TGamma[0][sigN:],TGamma[2][sigN:],c='r',s=100,alpha=0.7)

LinregressResult(slope=0.45040110614817025, intercept=0.5497210887139128, rvalue=0.3467025525001343, pvalue=0.0, stderr=0.0031963410075514388)
LinregressResult(slope=0.4624111751620011, intercept=0.5378280502777394, rvalue=0.3533491289298155, pvalue=0.0, stderr=0.003211319588371622)


In [12]:
kde = KDEMultivariate(Gscatt,var_type='cc', bw='normal_reference')
kdeTh = KDEMultivariate(TGscatt,var_type='cc', bw='normal_reference')

In [13]:
f,ax = pl.subplots(1,1)
xx,yy = np.meshgrid(np.linspace(0.8,1.2,100),np.linspace(0.8,1.2,100))
points = np.vstack([xx.ravel(), yy.ravel()])
z=kdeTh.pdf(points).reshape(xx.shape)
zc=kdeTh.cdf(points).reshape(xx.shape)
area = np.power(np.diff(xx)[0][0],2)
cdfArray = []
print z.sum()*area
CS = pl.contour(xx,yy,z,200,colors='k')
contourSet = CS.collections
cProb = 0
for iidxC,c in enumerate(contourSet):
    pth = c.get_paths()[0]
    if iidxC<len(contourSet)-1:
        nextPth = contourSet[iidxC+1].get_paths()[0]
        A_current = PolyArea(pth.vertices[:,0],pth.vertices[:,1])
        A_next = PolyArea(nextPth.vertices[:,0],nextPth.vertices[:,1])
    prob = kdeTh.pdf(pth.vertices).mean()
    probNext = kdeTh.pdf(nextPth.vertices).mean()
    pMass = (prob+probNext)*(A_current-A_next)/2
    cProb+=pMass
    cdfArray.append(cProb)
    #pl.scatter(iidxC,cProb,c='r')
#pl.axhline(5e-2,lw=1)
#pl.axhline(1e-2,lw=1)

0.9741407341547198


In [14]:
#pl.scatter(range(cdfArray.size),cdfArray)
#pl.axhline(5e-2,lw=1)
#pl.axhline(1e-2,lw=1)
cdfArray = np.array(cdfArray)
selContourIdx = []
for prob in [0.01,0.05,0.5,0.9]:
    selContourIdx.append(np.power((cdfArray-prob),2).argmin())
for idx in selContourIdx:
    #pl.scatter(idx+1,cdfArray[idx+1],c='r',s=50)
    pth = contourSet[idx+1].get_paths()[0]
    pl.plot(pth.vertices[:,0],pth.vertices[:,1])
pl.scatter(Gamma[0][sigN:],Gamma[2][sigN:],marker='x',c='k',s=50,zorder=20)

In [64]:
pkl.dump({'selectedContours':selContourIdx,'contours':contourSet},open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/shuffledGammaContours.pkl','wb'))

In [281]:
#pkl.dump(stList,open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/parrentData/FiringUnits-N.spk','wb'))

In [8]:
N = int(N)
reactivationStrengths  = np.array([Gamma[0][sigN:],Gamma[2][sigN:]])
TreactivationStrengths = np.array([TGamma[0][sigN:],TGamma[2][sigN:]])
fixedBinsName = str(100/Nslices)+'ms'
if Nslices==4:
    thBinsName = '4theta'
elif Nslices==1:
    thBinsName = 'theta'
pkl.dump(reactivationStrengths, open(WorkPath+animal+'-'+dday+'-'+fixedBinsName+'-vis.lsn','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(TreactivationStrengths,open(WorkPath+animal+'-'+dday+'-'+thBinsName+'-vis.lsn','wb'),pkl.HIGHEST_PROTOCOL)

Tidx = 1
Rz = []
threshold = []
for Midx in [0,2]:
    R0 = 0
    for ii in range(N):
        for jj in range(N):
            if ii != jj:
                R0 += Qs[Midx][ii]*corrMatrices[Tidx][ii,jj]*Qs[Midx][jj]
    #print R0
    Rz.append(R0)        
    threshold.append(R0.mean()+3.50*R0.std())
pkl.dump([sigN,Rz],open(WorkPath+animal+'-'+dday+'-'+fixedBinsName+'.R0','wb'),pkl.HIGHEST_PROTOCOL)
#######################
Rz = []
threshold = []
for Midx in [0,2]:
    R0 = 0
    for ii in range(N):
        for jj in range(N):
            if ii != jj:
                R0 += ThetaQs[Midx][ii]*ThetaCorrMatrices[Tidx][ii,jj]*ThetaQs[Midx][jj]
    Rz.append(R0)        
pkl.dump([sigN,Rz],open(WorkPath+animal+'-'+dday+'-'+thBinsName+'.R0','wb'),pkl.HIGHEST_PROTOCOL)
###############################
Rzl = []
for Midx in [0,2]:
    Ri = np.zeros(Qs[Midx][-10:].shape)
    for pIdx,pattern in enumerate(Ps[Tidx][-10:]):
        for ii in range(N):
            for jj in range(N):
                if ii != jj:
                    Ri[pIdx] += Qs[Midx][ii]*pattern[ii,jj]*Qs[Midx][jj]
    Rzl.append(Ri)        
pkl.dump([sigN,Rzl],open(WorkPath+animal+'-'+dday+'-'+fixedBinsName+'.Rl','wb'),pkl.HIGHEST_PROTOCOL)
#############################
Rzl = []
for Midx in [0,2]:
    Ri = np.zeros(ThetaQs[Midx][-10:].shape)
    for pIdx,pattern in enumerate(ThetaPs[Tidx][-10:]):
        for ii in range(N):
            for jj in range(N):
                if ii != jj:
                    Ri[pIdx] += ThetaQs[Midx][ii]*pattern[ii,jj]*ThetaQs[Midx][jj]
    Rzl.append(Ri)        
pkl.dump([sigN,Rzl],open(WorkPath+animal+'-'+dday+'-'+thBinsName+'.Rl','wb'),pkl.HIGHEST_PROTOCOL)
print 'D------------------------------------------------------NE!'

D------------------------------------------------------NE!


In [12]:
[z.size for z in Rz]

[36272, 27272]

In [255]:
xlist = np.linspace(-2.0, 2.0, 100) # Create 1-D arrays for x,y dimensions
ylist = np.linspace(-2.0, 2.0, 100)
X,Y = np.meshgrid(xlist, ylist) # Create 2-D grid xlist,ylist values
Z = np.sqrt(X**2 + Y**2)

In [256]:
sq = pl.contour(X, Y, Z, [0.5, 1.0, 1.2, 1.5,12], colors ='k')
pl.clabel(sq, inline=1, fontsize=10)

<a list of 4 text.Text objects>

In [15]:
pl.scatter(pth0.vertices[:,0],pth0.vertices[:,1])

NameError: name 'pth0' is not defined

In [19]:
savemat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/surrogateGamma.mat',pd.DataFrame(TGscatt,columns=['A_pre','A_post']).to_dict(orient='list'))

In [118]:
import statsmodels.api as sm
slopes = []
for _ii in range(5000):
    samp_idx = np.random.randint(0,Gscatt.shape[0],72)
    ols = sm.OLS(Gscatt[samp_idx,0],Gscatt[samp_idx,1]).fit()
    slopes.append(ols.params[0])
    #slopes.append(linregress(TGmSh[_ii],TGmSh[_ii+2]).slope)
sns.distplot(slopes,norm_hist=True)
pl.xlabel('Regression Slope')
pl.ylabel('Counts')
pkl.dump(slopes,open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/SlopesShuffled.pkl','wb'))

In [104]:
slope_kde = sm.nonparametric.KDEUnivariate(slopes)

In [116]:
slope_kde.fit()
pl.plot(slope_kde.support,slope_kde.density / slope_kde.density.sum())

In [114]:
np.sum(slope_kde.density/ slope_kde.density.sum())

0.9999999999999998